In [ ]:
import pandas as pd
df = pd.read_csv('data/3months-1min.csv', index_col='timestamp', parse_dates=True)
df

The "EMA death cross" describe a bearish signal that occurs when a shorter-term EMA in yellow, crosses below green

The "EMA golden cross" signals a potential shift in the trend from bearish to bullish, occurs when a longer-term EMA in green, crosses below yellow

In [ ]:
# Calculate the difference between the two EMAs
df['EMA_DIFF'] = df['EMA_50'] - df['EMA_200']
# Create a new column to store the signal
df['CROSS_TYPE'] = 0

df.head(5)


In [ ]:

# Calculate the difference between EMA_200 and EMA_50
df['EMA_DIFF'] = df['EMA_50'] - df['EMA_200']

# Find where the EMA_DIFF changes sign
crossings = (df['EMA_DIFF'].shift(1) < 0) & (df['EMA_DIFF'] > 0)
deaths = (df['EMA_DIFF'].shift(1) > 0) & (df['EMA_DIFF'] < 0)

# Assign signals to the DataFrame
df.loc[crossings, 'CROSS_TYPE'] = 1
df.loc[deaths, 'CROSS_TYPE'] = -1

# Count the number of golden and death crosses
number_golden = crossings.sum()
number_death = deaths.sum()

# Print the DataFrame with signals
print(f"number of golden crosses: {number_golden}, number of death crosses: {number_death}")



In [ ]:
crossing_segments = crossings.cumsum()
death_segments = deaths.cumsum()


In [ ]:
def plot_segment(df, index):
    from bokeh.plotting import figure, output_file, save
    from bokeh.models import ColumnDataSource

    source = ColumnDataSource(df)
    p = figure(x_axis_type='datetime', title='chart', width=1200, height=800)
    p.line(x='timestamp', y='close', source=source, line_width=2, color='black')
    # p.line(x='timestamp', y='high', source=source, line_width=2, color='red')
    # p.line(x='timestamp', y='high', source=source, line_width=2, color='blue')
    # p.line(x='timestamp', y='mid_band', source=source, line_width=0.5, color='purple')
    # p.line(x='timestamp', y='upper_band', source=source, line_width=2, color='blue')
    # p.line(x='timestamp', y='lower_band', source=source, line_width=2, color='blue')
    p.line(x='timestamp', y='EMA_200', source=source, line_width=2, color='green')
    p.line(x='timestamp', y='EMA_50', source=source, line_width=2, color='yellow')

    # Find intersection points for close line with upper_band and lower_band
    # for i in range(1, len(df)):
    #     # close line crossing upper_band
    #     if (df.loc[i - 1, 'close'] < df.loc[i - 1, 'upper_band'] and df.loc[i, 'close'] > df.loc[i, 'upper_band']) or (df.loc[i - 1, 'close'] > df.loc[i - 1, 'upper_band'] and df.loc[i, 'close'] < df.loc[i, 'upper_band']):
    #         p.circle(x=df.loc[i, 'timestamp'], y=df.loc[i, 'close'], color='magenta', size=6)
        
    #     # close line crossing lower_band
    #     if (df.loc[i - 1, 'close'] < df.loc[i - 1, 'lower_band'] and df.loc[i, 'close'] > df.loc[i, 'lower_band']) or (df.loc[i - 1, 'close'] > df.loc[i - 1, 'lower_band'] and df.loc[i, 'close'] < df.loc[i, 'lower_band']):
    #         p.circle(x=df.loc[i, 'timestamp'], y=df.loc[i, 'close'], color='cyan', size=6)

    #     # EMA_200 line crossing EMA_50
    #     if (df.loc[i - 1, 'EMA_200'] < df.loc[i - 1, 'EMA_50'] and df.loc[i, 'EMA_200'] > df.loc[i, 'EMA_50']) or (df.loc[i - 1, 'EMA_200'] > df.loc[i - 1, 'EMA_50'] and df.loc[i, 'EMA_200'] < df.loc[i, 'EMA_50']):
    #         p.circle(x=df.loc[i, 'timestamp'], y=df.loc[i, 'EMA_200'], color='red', size=6)

    # save the output to an HTML file
    output_file(f'./charts/chart-ema-{index}.html')
    # save the plot to the HTML file
    save(p)


In [ ]:
# Step 3: Calculate price change and time taken to start trending upward after a golden cross
golden_cross_data = []

for idx, row in df[df['CROSS_TYPE'] == 1].iterrows():
    i = df.index.get_loc(idx)
    upward_trend_started = False
    time_to_upward_trend = 0
    max_price = row['close']
    
    for j in range(i+1, len(df)):
        if df.iloc[j]['CROSS_TYPE'] == -1:
            break
        if not upward_trend_started and df.iloc[j]['close'] > row['close']:
            upward_trend_started = True
            time_to_upward_trend = j - i
        max_price = max(max_price, df.iloc[j]['close'])
    
    price_change = (max_price - row['close']) / row['close']
    golden_cross_data.append((idx, row['close'], max_price, price_change, time_to_upward_trend))

# Create a DataFrame with the golden cross data
golden_cross_df = pd.DataFrame(golden_cross_data, columns=['index', 'initial_close', 'max_close', 'price_change', 'time_to_upward_trend'])
golden_cross_df 

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(golden_cross_df)

# Create a new plot with a title and axis labels
p = figure(title="Golden Cross Data", x_axis_label='Index', y_axis_label='Price Change')

# Add a line renderer with legend and line thickness
p.line(x='index', y='price_change', source=source, legend_label='Price Change', line_width=2)

# Add a circle renderer with size, color, and alpha
p.circle(x='index', y='time_to_upward_trend', source=source, size=10, color='red', alpha=0.5)

# Add a legend to the plot
p.legend.location = "top_left"

# Save the plot as an HTML file and show it
output_file("./charts/golden_cross.html")
show(p)


In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

source = ColumnDataSource(golden_cross_df)

p = figure(x_axis_type='datetime', title='Golden Cross Data', width=1200, height=800)
p.vbar(x='index', top='price_change', source=source, color='blue', width=0.5)

p.xaxis.major_label_orientation = "vertical"

output_file("./charts/golden_cross2.html")
show(p)


In [ ]:
golden_cross_df.to_csv('data/golden-cross-data.csv', index=True)